### GPU or CPU Test

In [1]:
import os

# Anaconda ortamının (ai_env) kurulu olduğu ana dizini buluyoruz
conda_prefix = os.environ.get('CONDA_PREFIX')

if conda_prefix:
    # NVIDIA CUDA ve cuDNN .dll dosyalarının saklandığı klasör
    library_bin = os.path.join(conda_prefix, 'Library', 'bin')
    
    # Python'a "Bu klasörün içine de bak" diyoruz
    if os.path.exists(library_bin):
        os.add_dll_directory(library_bin)
        print(f"Başarıyla eklendi: Windows artık {library_bin} klasörünü görüyor.")
else:
    print("UYARI: CONDA_PREFIX bulunamadı. Lütfen Anaconda ortamında olduğundan emin ol.")

# Yolları tanıttıktan SONRA TensorFlow'u çağırıyoruz
import tensorflow as tf
print("Bulunan GPU Sayısı: ", len(tf.config.list_physical_devices('GPU')))

Başarıyla eklendi: Windows artık C:\Users\alkay\miniconda3\envs\ai_env\Library\bin klasörünü görüyor.
Bulunan GPU Sayısı:  1


In [1]:
import tensorflow as tf

# GPU'ları listele
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print(f"Harika! GPU Bulundu: {gpus[0]}")
    # Belleğin sadece gerektiği kadar kullanılmasını sağla (VRAM işgalini önler)
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("Bellek büyümesi (Memory Growth) aktif edildi.")
    except RuntimeError as e:
        print(e)
else:
    print("UYARI: GPU bulunamadı! İşlemler CPU'da yapılacak.")

UYARI: GPU bulunamadı! İşlemler CPU'da yapılacak.


In [2]:
import tensorflow as tf
print("Bulunan GPU Sayısı: ", len(tf.config.list_physical_devices('GPU')))

Bulunan GPU Sayısı:  0


### Blok 1: Kütüphaneler ve Ayarlar
Bu blok, projemiz için gereken tüm derin öğrenme (TensorFlow/Keras) ve görüntü işleme (OpenCV) kütüphanelerini içe aktarır. Ayrıca modelin eğitiminde kullanılacak LiDAR (Girdi) ve Maske (Çıktı/Ground Truth) verilerinin bilgisayardaki dosya yollarını tanımlar. Veri setinin train (eğitim) ve test olarak hangi indekslerden bölüneceği burada belirlenir.

In [6]:
import os
import cv2
import numpy as np
import tensorflow as tf
# Keras'ı doğrudan tf üzerinden çağıralım (Pylance bunu daha kolay anlar)
from tensorflow import keras
layers = keras.layers
models = keras.models
applications = keras.applications

import matplotlib.pyplot as plt
import sklearn.metrics as metrics # sklearn kuruluysa sarı çizgi gitmeli

# Dosya yolları
LIDAR_PATH = r"C:\Users\alkay\ITU_C\chactun\data\Chactun_ML_ready_lidar\lidar"
MASK_PATH = r"C:\Users\alkay\ITU_C\chactun\data\Chactun_ML_ready_masks\masks"

# Eğitim ve Test sınırları
TRAIN_END = 1764
TEST_END = 2093

### Blok 2: Veri Yükleyici (Data Generator)
Bu blok, donanım sınırlarını (RAM/VRAM) aşmamak için özel bir Veri Jeneratörü (`ChactunDataGenerator`) oluşturur. 
* Tüm veri setini RAM'e yüklemek yerine, eğitim sırasında görüntüleri belirlenen `batch_size` (yığın boyutu) kadar diskten anlık olarak çeker.
* Çözünürlüğü `480x480` olarak ayarlar. Yüksek çözünürlük RTX 4060'ın 8GB VRAM'ini zorlayacağı için `batch_size=2` olarak optimize edilmiştir.
* Siyah (0) olan maskeleri 1 (Pozitif Sınıf) olarak matematiksel olarak tersine çevirir (Invert).

In [7]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras

class ChactunDataGenerator(keras.utils.Sequence):
    def __init__(self, lidar_dir, mask_dir, start_idx, end_idx, batch_size=2, img_size=(480, 480)):
        self.lidar_dir = lidar_dir
        self.mask_dir = mask_dir
        self.indices = list(range(start_idx, end_idx + 1))
        self.batch_size = batch_size
        self.img_size = img_size

    def __len__(self):
        return int(np.ceil(len(self.indices) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        # X: (batch_size, 480, 480, 1), Y: (batch_size, 480, 480, 3)
        X = np.empty((len(batch_indices), *self.img_size, 1), dtype=np.float32)
        Y = np.empty((len(batch_indices), *self.img_size, 3), dtype=np.float32)

        for i, tile_idx in enumerate(batch_indices):
            # 1. LiDAR Görüntüsünü Oku ve İşle
            lidar_path = os.path.join(self.lidar_dir, f"tile_{tile_idx}_lidar.tif")
            lidar_img = cv2.imread(lidar_path, cv2.IMREAD_GRAYSCALE)
            
            if lidar_img is None:
                lidar_img = np.zeros(self.img_size, dtype=np.uint8)
                
            lidar_img = cv2.resize(lidar_img, self.img_size)
            X[i,] = np.expand_dims(lidar_img / 255.0, axis=-1)

            # 2. Maskeleri Oku
            m_building = cv2.imread(os.path.join(self.mask_dir, f"tile_{tile_idx}_mask_building.tif"), cv2.IMREAD_GRAYSCALE)
            m_aguada = cv2.imread(os.path.join(self.mask_dir, f"tile_{tile_idx}_mask_aguada.tif"), cv2.IMREAD_GRAYSCALE)
            m_platform = cv2.imread(os.path.join(self.mask_dir, f"tile_{tile_idx}_mask_platform.tif"), cv2.IMREAD_GRAYSCALE)

            if m_building is None: m_building = np.zeros(self.img_size, dtype=np.uint8)
            if m_aguada is None: m_aguada = np.zeros(self.img_size, dtype=np.uint8)
            if m_platform is None: m_platform = np.zeros(self.img_size, dtype=np.uint8)

            m_building = cv2.resize(m_building, self.img_size)
            m_aguada = cv2.resize(m_aguada, self.img_size)
            m_platform = cv2.resize(m_platform, self.img_size)

            combined_mask = np.stack([m_building, m_aguada, m_platform], axis=-1)
            Y[i,] = 1.0 - (combined_mask / 255.0)

        return X, Y


# Jeneratörleri oluşturuyoruz! (480x480 yüksek çözünürlük için Batch Size 2 yapıldı)
print("Eğitim ve Test Jeneratörleri hazırlanıyor...")

# Buradaki 8'leri silip 2 yaptık!
train_gen = ChactunDataGenerator(LIDAR_PATH, MASK_PATH, 0, TRAIN_END, batch_size=2)
test_gen = ChactunDataGenerator(LIDAR_PATH, MASK_PATH, TRAIN_END + 1, TEST_END, batch_size=2)

print("Jeneratörler Hazır! Yüksek çözünürlüklü ve bellek dostu veri yükleme aktif.")

Eğitim ve Test Jeneratörleri hazırlanıyor...
Jeneratörler Hazır! Yüksek çözünürlüklü ve bellek dostu veri yükleme aktif.


### Blok 3: VGG-19 Tabanlı U-Net Modeli (480x480)
Bu blok, yapay zekanın beyin yapısını (mimariyi) oluşturur. 
* **Encoder (Sol Kol):** Görüntüden özellikleri (kenar, köşe, şekil) çıkarmak için ImageNet üzerinde önceden eğitilmiş VGG-19 modeli kullanılır (Transfer Learning). 1 kanallı LiDAR verisi, modelin uyumluluğu için 3 kanala kopyalanır.
* **Decoder (Sağ Kol):** Çıkarılan bu özellikler, U-Net mimarisine özgü "Yukarı Örnekleme" (UpSampling) ve "Atlama Bağlantıları" (Skip Connections) ile tekrar orijinal çözünürlüğüne (480x480) büyütülür.
* **Çıkış:** Görüntüdeki her bir pikselin Bina, Su Rezervuarı veya Platform olma ihtimalini bağımsız olarak hesaplamak için 3 kanallı `Sigmoid` aktivasyon katmanıyla biter.

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models, applications

# DİKKAT: Giriş boyutu artık 480x480!
def build_vgg19_unet(input_shape=(480, 480, 1)):
    inputs = layers.Input(shape=input_shape)
    
    x_3ch = layers.Concatenate()([inputs, inputs, inputs])
    base_vgg = applications.VGG19(include_top=False, weights='imagenet', input_tensor=x_3ch)
    
    # Skip Connections (480x480'e göre otomatik boyutlanır)
    s1 = base_vgg.get_layer("block1_conv2").output 
    s2 = base_vgg.get_layer("block2_conv2").output 
    s3 = base_vgg.get_layer("block3_conv4").output 
    s4 = base_vgg.get_layer("block4_conv4").output 
    
    bridge = base_vgg.get_layer("block5_conv4").output 
    
    def upsample_block(x, skip, filters):
        x = layers.UpSampling2D((2, 2))(x)
        x = layers.Concatenate()([x, skip])
        x = layers.Conv2D(filters, 3, activation="relu", padding="same")(x)
        x = layers.Conv2D(filters, 3, activation="relu", padding="same")(x)
        return x

    d1 = upsample_block(bridge, s4, 512)
    d2 = upsample_block(d1, s3, 256)
    d3 = upsample_block(d2, s2, 128)
    d4 = upsample_block(d3, s1, 64)
    
    outputs = layers.Conv2D(3, 1, activation="sigmoid")(d4)
    return models.Model(inputs, outputs, name="VGG19_UNET_480")

print("VGG-19 U-Net Modeli (480x480) oluşturuluyor...")
model = build_vgg19_unet(input_shape=(480, 480, 1))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
    loss="binary_crossentropy", 
    metrics=["accuracy"]
)
model.summary()

VGG-19 U-Net Modeli (480x480) oluşturuluyor...
Model: "VGG19_UNET_480"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 480, 480, 1  0           []                               
                                )]                                                                
                                                                                                  
 concatenate_5 (Concatenate)    (None, 480, 480, 3)  0           ['input_2[0][0]',                
                                                                  'input_2[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 block1_conv1 (Conv2D)          (None,

### Blok 4: Model Eğitimi (Training) ve Asistanlar (Callbacks)
Bu blok, hazırlanan veri (Blok 2) ile model iskeletini (Blok 3) birleştirip öğrenme sürecini başlatır.
Eğitim sürecini akıllıca yönetmek için 3 asistan (callback) kullanılır:
1. **ModelCheckpoint:** Her epoch (tur) sonunda sadece başarısı artan en iyi modeli kaydeder.
2. **EarlyStopping:** Model belirli bir süre (10 tur) gelişmezse, boşa vakit/enerji harcamamak için eğitimi erken durdurur.
3. **ReduceLROnPlateau:** Model öğrenmekte zorlanmaya başladığında (takıldığında) adım boyutunu (learning rate) küçülterek daha hassas öğrenmesini sağlar.

In [9]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# En iyi modelin kaydedileceği klasörü oluşturalım
os.makedirs("checkpoints", exist_ok=True)

# Eğitim Asistanları (Callbacks)
# 1. Her epoch sonunda sadece en iyi modeli (val_loss'a göre) kaydeder
checkpoint = ModelCheckpoint(
    filepath="checkpoints/vgg19_unet_best.h5",
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

# 2. Model 10 tur boyunca ilerleme kaydetmezse eğitimi erken bitirir
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# 3. Eğitim duraklarsa öğrenme oranını (learning rate) düşürerek ince ayar yapar
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

# Eğitimi Başlatıyoruz!
EPOCHS = 50
print("Eğitim süreci başlatılıyor... RTX 4060 devreye giriyor 🚀")

history = model.fit(
    train_gen,                   # Blok 2'deki eğitim jeneratörü
    validation_data=test_gen,    # Blok 2'deki test (doğrulama) jeneratörü
    epochs=EPOCHS,
    callbacks=[checkpoint, early_stop, reduce_lr],
    verbose=1
)

Eğitim süreci başlatılıyor... RTX 4060 devreye giriyor 🚀
Epoch 1/50
 17/883 [..............................] - ETA: 2:28:30 - loss: 0.1482 - accuracy: 0.0699

KeyboardInterrupt: 